In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import glob

df = pd.read_csv("sampled_game_Reviews.csv")
print(f"Total data: {len(df)} baris")

Couldn't import dot_parser, loading of dot files will not be possible.
Total data: 369 baris


In [ ]:
# Ganti NaN jadi string kosong, lalu hitung panjangnya
df['char_count'] = df['Review'].fillna("").apply(len)

# Hitung rata-rata
average_chars = df['char_count'].mean()

print(f"Rata-rata karakter per Review: {average_chars:.2f}")


Rata-rata karakter per review: 358.59


In [ ]:
# 2. Pastikan kolom 'Reviews' ada dan tidak berisi NaN
df = df.dropna(subset=["Review"])

# 3. Ambil 10 sampel acak
sample_Reviews = df["Review"].sample(n=10, random_state=42).reset_index(drop=True)

# 4. Tampilkan hasil
print(sample_Reviews)

0    Garbage game, they ripped out the enjoyment of...
1    incredible  beautiful and quality  game a must...
2    My dislike for this game is more of a failure ...
3                                         perfect game
4    This game is a hidden gem. Carefully craft you...
5                                                 jeps
6    Attempted to sue an indie dev for making a bet...
7    Roguelite, roguelike, Borderlands with coop-mu...
8    This one is another in-between game for me. Ar...
9                           ah heck i hate key hunting
Name: Review, dtype: object


In [4]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Sekarang bisa download
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/divaoncom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divaoncom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Install hanya sekali
!pip install nltk spacy
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 296.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.4/634.4 kB 282.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.9/844.9 kB 319.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 292.5 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 280.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 278.1 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 

In [7]:
import pandas as pd
import re
import nltk
import spacy

# Load data
df = pd.read_csv("sampled_game_Reviews.csv")
df['Review'] = df['Review'].astype(str)

# Tampilkan jumlah data
print("Jumlah data:", len(df))
df[['Review']].head()


Jumlah data: 369


,Review
0,My only major negative is the story. All the c...
1,"MEH, big meh. Bland storyline, Japanese melodr..."
2,Honestly? Just insane. I used to play Ace Comb...
3,I've never played anything from this series be...
4,I'm on the highway to the danger zone


In [8]:
print("Jumlah data sebelum Case Folding:", df['Review'].notnull().sum())

df['case_folding'] = df['Review'].str.lower()

print("Jumlah data sesudah Case Folding:", df['case_folding'].notnull().sum())
print("Contoh:\nSebelum:", df['Review'][0], "\nSesudah:", df['case_folding'][0])


Jumlah data sebelum Case Folding: 369
Jumlah data sesudah Case Folding: 369
Contoh:
Sebelum: My only major negative is the story. All the characters are lame and everything comes across as incredibly campy. It is a fun game otherwise. Every mission is unique, with the only noticeably bad one being #20. 
Sesudah: my only major negative is the story. all the characters are lame and everything comes across as incredibly campy. it is a fun game otherwise. every mission is unique, with the only noticeably bad one being #20.


In [9]:
print("Jumlah data sebelum Cleansing:", df['case_folding'].notnull().sum())

df['cleansing'] = df['case_folding'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

print("Jumlah data sesudah Cleansing:", df['cleansing'].notnull().sum())
print("Contoh:\nSebelum:", df['case_folding'][0], "\nSesudah:", df['cleansing'][0])


Jumlah data sebelum Cleansing: 369
Jumlah data sesudah Cleansing: 369
Contoh:
Sebelum: my only major negative is the story. all the characters are lame and everything comes across as incredibly campy. it is a fun game otherwise. every mission is unique, with the only noticeably bad one being #20. 
Sesudah: my only major negative is the story all the characters are lame and everything comes across as incredibly campy it is a fun game otherwise every mission is unique with the only noticeably bad one being 


In [10]:
normalization_dict = {
    "u": "you", "r": "are", "ur": "your", "pls": "please", "plz": "please",
    "thx": "thanks", "ty": "thank you", "btw": "by the way", "lol": "laughing out loud",
    "lmao": "laughing my ass off", "omg": "oh my god", "wtf": "what the fuck",
    "idk": "i do not know", "imo": "in my opinion", "imho": "in my humble opinion",
    "afaik": "as far as i know", "brb": "be right back", "bbl": "be back later",
    "gg": "good game", "ez": "easy", "nerf": "weaken", "buff": "strengthen",
    "noob": "newbie", "grind": "repetitive task", "op": "overpowered",
    "laggy": "slow connection", "gitgud": "get good", "f2p": "free to play",
    "p2w": "pay to win", "dlc": "downloadable content", "npc": "non player character",
    "fps": "frames per second", "afk": "away from keyboard", "xp": "experience points",
    "lvl": "level", "bossfight": "boss fight", "rng": "random number generator",
    "camping": "staying in one spot", "gank": "ambush attack", "re": "regarding",
    "af": "as fuck", "xd": "bad", "f": "fuck", "ive": "i have", "rp": "roleplay",
    "fckin": "fucking", "tl": "too long", "dr": "didnt read", "lcg": "living card game", 
    "stori": "story", "differ": "different"
}

def normalize(text):
    return ' '.join([normalization_dict.get(word, word) for word in text.split()])

print("Jumlah data sebelum Normalization:", df['cleansing'].notnull().sum())

df['normalization'] = df['cleansing'].apply(normalize)

print("Jumlah data sesudah Normalization:", df['normalization'].notnull().sum())
print("Contoh:\nSebelum:", df['cleansing'][0], "\nSesudah:", df['normalization'][0])


Jumlah data sebelum Normalization: 369
Jumlah data sesudah Normalization: 369
Contoh:
Sebelum: my only major negative is the story all the characters are lame and everything comes across as incredibly campy it is a fun game otherwise every mission is unique with the only noticeably bad one being  
Sesudah: my only major negative is the story all the characters are lame and everything comes across as incredibly campy it is a fun game otherwise every mission is unique with the only noticeably bad one being


In [11]:
def tokenize(text):
    return text.split()

print("Jumlah data sebelum Tokenizing:", df['normalization'].notnull().sum())

df['tokenized'] = df['normalization'].apply(tokenize)

print("Jumlah data sesudah Tokenizing:", df['tokenized'].notnull().sum())
print("Contoh:\nSebelum:", df['normalization'][0], "\nSesudah:", df['tokenized'][0])


Jumlah data sebelum Tokenizing: 369
Jumlah data sesudah Tokenizing: 369
Contoh:
Sebelum: my only major negative is the story all the characters are lame and everything comes across as incredibly campy it is a fun game otherwise every mission is unique with the only noticeably bad one being 
Sesudah: ['my', 'only', 'major', 'negative', 'is', 'the', 'story', 'all', 'the', 'characters', 'are', 'lame', 'and', 'everything', 'comes', 'across', 'as', 'incredibly', 'campy', 'it', 'is', 'a', 'fun', 'game', 'otherwise', 'every', 'mission', 'is', 'unique', 'with', 'the', 'only', 'noticeably', 'bad', 'one', 'being']


In [14]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 301.9 kB/s eta 0:00:0000:0100:02
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 330.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.0/779.0 kB 443.6 kB/s eta 0:00:00a 0:00:01
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 255.9 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.3.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.6
    Uninstalling thinc-8.3.6:
      Successfully un

In [33]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
!python -m spacy download en_core_web_sm

zsh:1: command not found: python


In [35]:
nlp = spacy.load("en_core_web_sm")

def lemmatize(tokens):
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

print("Jumlah data sebelum Lemmatization:", df['tokenized'].notnull().sum())

df['lemmatized'] = df['tokenized'].apply(lemmatize)

print("Jumlah data sesudah Lemmatization:", df['lemmatized'].notnull().sum())
print("Contoh:\nSebelum:", df['tokenized'][0], "\nSesudah:", df['lemmatized'][0])


Jumlah data sebelum Lemmatization: 369
Jumlah data sesudah Lemmatization: 369
Contoh:
Sebelum: ['my', 'only', 'major', 'negative', 'is', 'the', 'story', 'all', 'the', 'characters', 'are', 'lame', 'and', 'everything', 'comes', 'across', 'as', 'incredibly', 'campy', 'it', 'is', 'a', 'fun', 'game', 'otherwise', 'every', 'mission', 'is', 'unique', 'with', 'the', 'only', 'noticeably', 'bad', 'one', 'being'] 
Sesudah: ['my', 'only', 'major', 'negative', 'be', 'the', 'story', 'all', 'the', 'character', 'be', 'lame', 'and', 'everything', 'come', 'across', 'as', 'incredibly', 'campy', 'it', 'be', 'a', 'fun', 'game', 'otherwise', 'every', 'mission', 'be', 'unique', 'with', 'the', 'only', 'noticeably', 'bad', 'one', 'be']


In [36]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

print("Jumlah data sebelum Stopword Removal:", df['lemmatized'].notnull().sum())

df['final_tokens'] = df['lemmatized'].apply(remove_stopwords)

print("Jumlah data sesudah Stopword Removal:", df['final_tokens'].notnull().sum())
print("Contoh:\nSebelum:", df['lemmatized'][0], "\nSesudah:", df['final_tokens'][0])


Jumlah data sebelum Stopword Removal: 369
Jumlah data sesudah Stopword Removal: 369
Contoh:
Sebelum: ['my', 'only', 'major', 'negative', 'be', 'the', 'story', 'all', 'the', 'character', 'be', 'lame', 'and', 'everything', 'come', 'across', 'as', 'incredibly', 'campy', 'it', 'be', 'a', 'fun', 'game', 'otherwise', 'every', 'mission', 'be', 'unique', 'with', 'the', 'only', 'noticeably', 'bad', 'one', 'be'] 
Sesudah: ['major', 'negative', 'story', 'character', 'lame', 'everything', 'come', 'across', 'incredibly', 'campy', 'fun', 'game', 'otherwise', 'every', 'mission', 'unique', 'noticeably', 'bad', 'one']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divaoncom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
import pandas as pd

# Asumsi df sudah dimuat dari file CSV sebelumnya

# Gabungkan hasil akhir ke dalam kolom teks baru (cleaned_Reviews)
df['cleaned_Reviews'] = df['final_tokens'].apply(lambda x: ' '.join(x))

# Pilih dan urutkan kolom yang akan disimpan
output_columns = [
    'Game',               # nama game (pastikan kolom ini ada di file asli)
    'Review',             # Review asli
    'case_folding',
    'cleansing',
    'normalization',
    'tokenized',
    'lemmatized',
    'final_tokens',       # hasil setelah stopword removal
    'cleaned_Reviews'     # versi string final
]

# Periksa apakah kolom 'game' ada, jika tidak, beri peringatan
if 'game' not in df.columns:
    print("⚠️ Kolom 'game' tidak ditemukan! Pastikan ada di file CSV asli.")
    if 'game' in output_columns:
        output_columns.remove('game')

# Simpan ke CSV
df[output_columns].to_csv("preprocessed_Reviews.csv", index=False)

print("✅ File berhasil disimpan sebagai 'preprocessed_Reviews.csv'")


⚠️ Kolom 'game' tidak ditemukan! Pastikan ada di file CSV asli.
✅ File berhasil disimpan sebagai 'preprocessed_Reviews.csv'


In [40]:
df = pd.read_csv("preprocessed_Reviews.csv")
df.head(10)

,Game,Review,case_folding,cleansing,normalization,tokenized,lemmatized,final_tokens,cleaned_Reviews
0,ACE COMBAT™ 7: SKIES UNKNOWN,My only major negative is the story. All the c...,my only major negative is the story. all the c...,my only major negative is the story all the ch...,my only major negative is the story all the ch...,"['my', 'only', 'major', 'negative', 'is', 'the...","['my', 'only', 'major', 'negative', 'be', 'the...","['major', 'negative', 'story', 'character', 'l...",major negative story character lame everything...
1,ACE COMBAT™ 7: SKIES UNKNOWN,"MEH, big meh. Bland storyline, Japanese melodr...","meh, big meh. bland storyline, japanese melodr...",meh big meh bland storyline japanese melodrama...,meh big meh bland storyline japanese melodrama...,"['meh', 'big', 'meh', 'bland', 'storyline', 'j...","['meh', 'big', 'meh', 'bland', 'storyline', 'j...","['meh', 'big', 'meh', 'bland', 'storyline', 'j...",meh big meh bland storyline japanese melodrama...
2,ACE COMBAT™ 7: SKIES UNKNOWN,Honestly? Just insane. I used to play Ace Comb...,honestly? just insane. i used to play ace comb...,honestly just insane i used to play ace combat...,honestly just insane i used to play ace combat...,"['honestly', 'just', 'insane', 'i', 'used', 't...","['honestly', 'just', 'insane', 'I', 'use', 'to...","['honestly', 'insane', 'I', 'use', 'play', 'ac...",honestly insane I use play ace combat game ps ...
3,ACE COMBAT™ 7: SKIES UNKNOWN,I've never played anything from this series be...,i've never played anything from this series be...,ive never played anything from this series bef...,i have never played anything from this series ...,"['i', 'have', 'never', 'played', 'anything', '...","['I', 'have', 'never', 'play', 'anything', 'fr...","['I', 'never', 'play', 'anything', 'series', '...",I never play anything series one fun dogfighti...
4,ACE COMBAT™ 7: SKIES UNKNOWN,I'm on the highway to the danger zone,i'm on the highway to the danger zone,im on the highway to the danger zone,im on the highway to the danger zone,"['im', 'on', 'the', 'highway', 'to', 'the', 'd...","['I', 'm', 'on', 'the', 'highway', 'to', 'the'...","['I', 'highway', 'danger', 'zone']",I highway danger zone
5,ACE COMBAT™ 7: SKIES UNKNOWN,Best ace combat game since 4. Glad they return...,best ace combat game since 4. glad they return...,best ace combat game since glad they returned...,best ace combat game since glad they returned ...,"['best', 'ace', 'combat', 'game', 'since', 'gl...","['good', 'ace', 'combat', 'game', 'since', 'gl...","['good', 'ace', 'combat', 'game', 'since', 'gl...",good ace combat game since glad return strange...
6,Age of Fear 2: The Chaos Lord GOLD,I bought this game in a bundle together with A...,i bought this game in a bundle together with a...,i bought this game in a bundle together with a...,i bought this game in a bundle together with a...,"['i', 'bought', 'this', 'game', 'in', 'a', 'bu...","['I', 'buy', 'this', 'game', 'in', 'a', 'bundl...","['I', 'buy', 'game', 'bundle', 'together', 'ao...",I buy game bundle together aof game share engi...
7,Age of Fear 2: The Chaos Lord GOLD,Absolute travesty. The UI/UX is so clunky and ...,absolute travesty. the ui/ux is so clunky and ...,absolute travesty the uiux is so clunky and sl...,absolute travesty the uiux is so clunky and sl...,"['absolute', 'travesty', 'the', 'uiux', 'is', ...","['absolute', 'travesty', 'the', 'uiux', 'be', ...","['absolute', 'travesty', 'uiux', 'clunky', 'sl...",absolute travesty uiux clunky slow pure
8,Age of Wonders 4,Taking the best of AoW 3 and AoW Planetfall an...,taking the best of aow 3 and aow planetfall an...,taking the best of aow and aow planetfall and...,taking the best of aow and aow planetfall and ...,"['taking', 'the', 'best', 'of', 'aow', 'and', ...","['take', 'the', 'good', 'of', 'aow', 'and', 'a...","['take', 'good', 'aow', 'aow', 'planetfall', '...",take good aow aow planetfall add even customal...
9,Age of Wonders 4,They removed racial traits in exchange for oth...